In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(filepath_or_buffer='/kaggle/input/the-rise-of-ai-based-llms/The Rise of AI-Based Large Language Models (LLMs) - LLM data.csv', 
                skiprows=1,).drop(columns=['link'])
df['year'] = df['date'].apply(func=lambda x: int(x.split()[1]))
df['parameters (bn)'] = df['trained on x billion parameters'].apply(func=lambda x: float(x) if x.isnumeric() else np.nan)
df.head(n=100)

,name,owner,trained on x billion parameters,date,note / * = parameters undisclosed,year,parameters (bn)
0,BERT,Google,0.34,Oct 2018,NaN,2018,NaN
1,GPT-2,OpenAI,1.5,Feb 2019,trained on Reddit only,2019,NaN
2,T5,Google,11,Oct 2019,NaN,2019,11.0
3,Megatron-11B,Meta / Facebook,11,Apr 2020,NaN,2020,11.0
4,BlenderBot1,Meta / Facebook,9.4,Apr 2020,NaN,2020,NaN
...,...,...,...,...,...,...,...
75,Web LLM,Independent,7,Jun 2023,Browser-based LLM Chatbot,2023,7.0
76,OpenLLM,Independent,13,Jun 2023,NaN,2023,13.0
77,Ernie Bot 3.5,Baidu,200,July 2023,Surpassing ChatGPT (3.5) in comprehensive abil...,2023,200.0
78,Claude 2,Anthropic,52,July 2023,"Expanded input and output length (up to 100,00...",2023,52.0


In [2]:
from plotly.express import scatter
scatter(data_frame=df.sort_values(by='owner'), x='year', y='parameters (bn)', hover_name='name', color='owner')